In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial.distance import cdist
pd.options.mode.chained_assignment = None


In [48]:
'''Load and save functions.'''

filepath = r'scripts/bbb-new'

# load file
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    return pd.read_csv(f)

# save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '/' + "{}_NEW.txt".format(fName), sep =',', index=False, float_format="%.6f")
    return print("Saved {}_NEW.txt to {}".format(fName,filepath))

# load GTFS files
trips = load('trips')
stops = load('stops')
stop_times = load('stop_times')
shapes = load('shapes')
routes = load('routes')
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 9 columns):
route_id            22 non-null int64
agency_id           22 non-null int64
route_short_name    22 non-null object
route_long_name     22 non-null object
route_desc          0 non-null float64
route_type          22 non-null int64
route_url           22 non-null object
route_color         22 non-null object
route_text_color    22 non-null object
dtypes: float64(1), int64(3), object(5)
memory usage: 1.6+ KB


In [45]:
'''USED FOR TESTING- query a single trip, shape, stops'''
# get one trip to test 
# onetrip = trips[(trips.trip_id == 782006)]
# oneshape = shapes[(shapes.shape_id == 23897)]
# stopTimeStops = stop_times['stop_id'][(stop_times.trip_id == 782006)].tolist()
# # stopTimeStops = (stop_times.trip_id == 782006).tolist()
# neededStops = stops[stops['stop_id'].isin(stopTimeStops)]
# lstlat = neededStops['stop_lat'].tolist()
# lstlon = neededStops['stop_lon'].tolist()
# stpCoord = np.array(list(zip(lstlat,lstlon)))
# stopcoordslst = list(result)
# stopsNstop_times.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled',
       'timepoint', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat',
       'stop_lon', 'zone_id', 'stop_url', 'location_type', 'parent_station',
       'stop_timezone', 'wheelchair_boarding'],
      dtype='object')

In [36]:
# stop_times.info()
'''Need to create a massive DF with the stop_times, shape_ids, stops. Then the distance function and the rest of the script'''

# join shapes and trips
# seems like there are more than one trip per shape... how can this be joined w/o creatng a huge df 
shapesNtrips = pd.merge(shapes,trips,how='left',on='shape_id',suffixes=('','_trip'))
shapesNtrips = shapesNtrips.drop(['shape_dist_traveled', 'service_id','trip_headsign', 'trip_short_name', 'direction_id', 'block_id','wheelchair_accessible', 'bikes_allowed','block_id'],axis=1)
shapesNtripsSorted = shapesNtrips.sort_values(['trip_id','shape_pt_sequence'])

In [50]:
shapesNtripsSorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051645 entries, 629 to 1050108
Data columns (total 6 columns):
shape_id             1051645 non-null int64
shape_pt_lat         1051645 non-null float64
shape_pt_lon         1051645 non-null float64
shape_pt_sequence    1051645 non-null int64
route_id             1051645 non-null int64
trip_id              1051645 non-null int64
dtypes: float64(2), int64(4)
memory usage: 56.2 MB


In [37]:
# join stop lat longs to stop_times
stopsNstop_times = pd.merge(stop_times,stops,how='left',on='stop_id')
stopsNstop_times = stopsNstop_times.drop(['arrival_time', 'departure_time','stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled', 'timepoint', \
                        'stop_code', 'stop_name', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station','stop_timezone', 'wheelchair_boarding'],axis=1)
stopsNstop_timesSorted = stopsNstop_times.sort_values(['trip_id','stop_sequence'])


In [ ]:
subStopTimes.info()

In [111]:
'''have the input accept one route'''
# singleRouteId = sys.argv[0]
# singleRouteId = routes.route_id == 3025
singleRoute = trips.route_id == 3025

subShapeList = trips[singleRoute].shape_id.unique().tolist()
subShapes = shapes[shapes.shape_id.isin(subShapeList)]

subTripsList = trips[singleRoute].trip_id.unique().tolist()
subTrips = trips[trips.trip_id.isin(subTripsList)]

subStopTimesList = stop_times.stop_id[stop_times.trip_id.isin(subTripsList)].tolist()

subStops = stops[stops.stop_id.isin(subStopTimesList)]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 20 to 872
Data columns (total 12 columns):
stop_id                30 non-null int64
stop_code              30 non-null int64
stop_name              30 non-null object
stop_desc              30 non-null object
stop_lat               30 non-null float64
stop_lon               30 non-null float64
zone_id                0 non-null float64
stop_url               0 non-null float64
location_type          0 non-null float64
parent_station         0 non-null float64
stop_timezone          0 non-null float64
wheelchair_boarding    30 non-null int64
dtypes: float64(7), int64(3), object(2)
memory usage: 3.0+ KB


In [114]:
subShapes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 19511 to 19645
Data columns (total 5 columns):
shape_id               135 non-null int64
shape_pt_lat           135 non-null float64
shape_pt_lon           135 non-null float64
shape_pt_sequence      135 non-null int64
shape_dist_traveled    135 non-null float64
dtypes: float64(3), int64(2)
memory usage: 6.3 KB


In [121]:
'''MAIN WORK HERE'''

# turn stops coords to list, then numpy array
# this will be used for the distance function
stpLat = subStops.stop_lat.tolist()
stpLon = subStops.stop_lon.tolist()
stpCoord = np.array(list(zip(stpLat,stpLon)))

# turn shape coords to list, then turn to numpy array
# this will be used for the distance function
shpLat = subShapes.shape_pt_lat.tolist()
shpLon = subShapes.shape_pt_lon.tolist()
shpCoord = np.array(list(zip(shpLat,shpLon)))

# create a dataframe to use as a way to identify the rows that need to be changed 
# shapes = pd.DataFrame({
#     'shape_id' : shapes.shape_id,
#     'shape_pt_lat' : shapes.shape_pt_lat,
#     'shape_pt_lon' : shapes.shape_pt_lon,
#     'shape_pt_sequence' : shapes.shape_pt_sequence,
#     'shape_dist_traveled' : shapes.shape_dist_traveled,
#     # use the euclidean distance function here and multiply by 10000 to get some eaiser numbers to work with
#     'dist_to_stp': cdist(shpCoord,stpCoord,'euclidean').min(axis=1) * 10000

# })

# shapes['dist_to_stp'] = cdist(shpCoord,stpCoord,'euclidean').min(axis=1) * 10000
subShapes['dist_to_stp'] = cdist(shpCoord,stpCoord,'euclidean').min(axis=1) * 10000


''''''

# find difference between dist_to_stp rows
subShapes['diff'] = subShapes['dist_to_stp'] - subShapes['dist_to_stp'].shift(1)
# create a dummy column marking where the diff is negative
subShapes['dummy'] = np.where(subShapes['diff'] < 0,'1','0')
'''Keep where diff == NaN- this is first shape point and 'should' begin at a stop OR where distance to stop is < 2.8 OR where dummy == 1 AND the the value below it is == 0
AND Keep where dummy == 1 AND distance to stop is < 5 (this may change). This logic will ensure the closest stop is grabbed based on criteria other than spatial criteria. 
The result is a column named "keep" with values of either keep or throw. These values are used later on with a buffer from the stop to highlight the rows to change'''
subShapes['keep'] = np.where(np.logical_or(subShapes['diff'].isna(),\
                                                     np.logical_or(subShapes['dist_to_stp'] < 2.8,\
                                                                    np.logical_and(np.logical_and(subShapes['dummy'] == '1', subShapes['dist_to_stp'] < 5.),\
                                                                                   np.logical_and(subShapes['dummy'] == '1', subShapes['dummy'].shift(-1) == '0')\
                                                                                  )\
                                                                  )\
                                                    ) ,'keep','throw')


'''With all of the other parameters above, have a column with a buffer of the stop and check to see if a shape point marked as "keep"
falls within that buffer. If it does, update the shape point lat and lon for that row. There may be more than one shape point moved, inital testng 
has shown this not to be a problem. The generaliztion around the stops "should" not be noticeable. More testing should be done to 100% confirm'''

# create a geodataframe to do an intersection 
intersect_df = gpd.GeoDataFrame(subShapes, geometry = gpd.points_from_xy(subShapes.shape_pt_lon,subShapes.shape_pt_lat))
intersect_df['shape_pt_geometry'] = intersect_df['geometry']
intersect_df.drop(['geometry'],axis=1)

# stop dataframe with a buffer- distance can be adjusted if need be
stopdf = gpd.GeoDataFrame(subStops, geometry=gpd.points_from_xy(subStops.stop_lon,subStops.stop_lat))
stopdf['geometry'] = stopdf.geometry.buffer(.0005)

# intersect df based on buffer polygon
intersect_join = gpd.sjoin(intersect_df,stopdf,how='inner',op='intersects')
# only keep the records 'keep'
intersect_join = intersect_join[(intersect_join.keep == 'keep')]

'''join the intersect df back to the nearest shape df based on index and create the new shapes.txt'''
badMerge = pd.merge(subShapes,intersect_join,how='left',suffixes=('','_y'),left_index=True,right_index=True)
# got a duplicated index somehow, drop it here
finalMerge = badMerge[~badMerge.index.duplicated()]

# update the new join shape lat lon with stop lat lon based on the 'keep' column
finalMerge.loc[finalMerge['keep'] == 'keep', 'shape_pt_lat'] = finalMerge['stop_lat']
finalMerge.loc[finalMerge['keep'] == 'keep', 'shape_pt_lon'] = finalMerge['stop_lon']

# merge the lat lon 
finalMerge.round({'shape_pt_lat': 6, 'shape_pt_lon': 6})

# drop unwanted columns from the final merge df to create the shapes_NEW.txt
dropCols = [i for i in range(len(finalMerge.columns)) if i > 4]
finalMerge.drop(finalMerge.columns[dropCols],axis=1,inplace=True)
# finalMerge.info()

#  print the amount of points that were changed
changedPoints = finalMerge.shape_pt_lat != subShapes.shape_pt_lat
print('SWEET! You modified {} shape points.\n'.format(len(finalMerge[changedPoints])))

# finalMerge.head()
save(finalMerge,'shapes')
# this is the final product- OLD WAY, keep this for testing
# finalMerge[['shape_id','shape_pt_lat','shape_pt_lon', 'shape_pt_sequence']]
# finalMerge.info()


SWEET! You modified 35 shape points.

Saved shapes_FIXED.txt to scripts/bbb-new


In [37]:
finalMerge.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20808 entries, 0 to 20807
Data columns (total 5 columns):
shape_id               20808 non-null int64
shape_pt_lat           20808 non-null float64
shape_pt_lon           20808 non-null float64
shape_pt_sequence      20808 non-null int64
shape_dist_traveled    20808 non-null float64
dtypes: float64(3), int64(2)
memory usage: 1.6 MB


In [ ]:
'''Test graph to look at the distances of the shape points in comparison to stops along the route'''
# plot the distance from the stop... the 'valleys' are where the stops are
# plt.figure()
# plt.ylabel("Distance From Stop").set_color("White")
# nearest_shapePnt_df['dist_to_stp'].plot(kind='bar',figsize=(20,7)).axes.get_xaxis().set_visible(False)
# plt.show()

In [ ]:
'''Test map (graph) to look at ALL of the shape points for a route in comparison to stops along the route'''
# plot the stops (red) and the shape points the script returns (blue)
# all of the shape points FOR TESTING
# allShapePoints = gpd.GeoDataFrame(oneshape, geometry=gpd.points_from_xy(oneshape.shape_pt_lon,oneshape.shape_pt_lat))
# stopsTest = testStopdf.plot(figsize=(15,15),color='red', marker='o',markersize=150)
# # allShapePoints.plot(figsize=(15,15),color='blue')
# allShapePoints.plot(ax=stopsTest, color='blue',marker='o',markersize=20)
# plt.show()




In [ ]:
'''Test map (graph) to look at a subset of the shape points identifed as being nearest to stops along the route'''
# # testPointsdf = gpd.GeoDataFrame(oneshape, geometry=gpd.points_from_xy(oneshape.shape_pt_lon,oneshape.shape_pt_lat))
# testPointsdf = gpd.GeoDataFrame(keepPnts, geometry=gpd.points_from_xy(keepPnts.shape_pt_lon,keepPnts.shape_pt_lat))
# testPointsdf
# testStopdf = gpd.GeoDataFrame(neededStops, geometry=gpd.points_from_xy(neededStops.stop_lon,neededStops.stop_lat))
# testStopdf['geometry'] = testStopdf.geometry.buffer(0.0010)
# testStopdf
#plot shape points and stops... see an extra shape point near southern part of route
# testPointsdf = gpd.GeoDataFrame(keepPnts, geometry=gpd.points_from_xy(keepPnts.shape_pt_lon,keepPnts.shape_pt_lat))
# newShapesGdf = gpd.GeoDataFrame(newShapesTxt, geometry=gpd.points_from_xy(newShapesTxt.shape_pt_lon,newShapesTxt.shape_pt_lat))
# stopsTest = testStopdf.plot(figsize=(15,15),color='red',marker='o',markersize=150)
# newShapesGdf.plot(ax=stopsTest, color='blue',marker='o',markersize=20)
# plt.show()
    